In [ ]:
#imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("darkgrid")

In [ ]:
#read in data
data = pd.read_excel('DonationsSDNHM.xlsx', sheet_name = "Donation Information")

In [ ]:
#EDA

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
column_descriptions = pd.read_excel('DonationsSDNHM.xlsx', sheet_name = 'Donation Informatin Fields')

In [ ]:
column_descriptions #good to use as a reference

In [ ]:
number_na = data.isna()

In [ ]:
number_na.sum()

In [ ]:
ratio_missing = number_na.sum()/len(data)

In [ ]:
ratio_missing

In [ ]:
sns.heatmap(data.isna(), cbar = False)

In [ ]:
data['Zip Code'].value_counts()

In [ ]:
donationstats = data['Recognition Amount'].describe()

In [ ]:
donationstats = pd.DataFrame(donationstats)

In [ ]:
donationstats

In [ ]:
#donationstats.to_excel('donatoinstats.xlsx')

In [ ]:
donations = sns.violinplot(x=data["Recognition Amount"])

In [ ]:
def categorical_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', verbose=True):
    '''
    Helper function that gives a quick summary of a given column of categorical data
    Arguments
    =========
    dataframe: pandas dataframe
    x: str. horizontal axis to plot the labels of categorical data, y would be the count
    y: str. vertical axis to plot the labels of categorical data, x would be the count
    hue: str. if you want to compare it another variable (usually the target variable)
    palette: array-like. Colour of the plot
    Returns
    =======
    Quick Stats of the data and also the count plot
    '''
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = dataframe[column_interested]
    print("***Descrition of Values***")
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print("***Total value counts of the series***")
        print(series.value_counts())
        print("***Ratio of values with respect to the total series***")
        print(series.value_counts()/(len(series)))

    sns.countplot(x=x, y=y, hue=hue, data=dataframe, palette="RdBu_r")
    plt.show()


In [ ]:
categorical_summarized(data, y='Designation')

In [ ]:
data.Designation.value_counts()

In [ ]:
data.columns

In [ ]:
data.Amount.head(20)

In [ ]:
data[["Recognition Amount","Amount"]]

In [ ]:
high_don = data[data.Amount >= 1000]

In [ ]:
high_don.head(9)

In [ ]:
high_donations = sns.violinplot(x=high_don["Recognition Amount"])

In [ ]:
progressors = data[(data.Amount >= 100) & (data.Amount <= 1000)] #creating a subset of the donors who could progress

In [ ]:
progressors_plot = sns.violinplot(x=progressors["Recognition Amount"])

In [ ]:
progressors.head()

In [ ]:
distofprog = progressors.groupby('Recognition Lookup ID')['Recognition Lookup ID'].count()

In [ ]:
ax = sns.barplot(data=distofprog)

In [ ]:
doncount = pd.DataFrame({'count': progressors.groupby('Recognition Lookup ID')['Recognition Lookup ID'].count()}).reset_index()

In [ ]:
doncount.head()

In [ ]:
doncount.sort_values('count',ascending = False);

In [ ]:
progressors.columns

In [ ]:
progressors.groupby(['Recognition Lookup ID', 'Date']).size()

In [ ]:
data.head();

In [ ]:
data.rename(columns={'Recognition Lookup ID':'ID','Recognition Name':'Name'},inplace=True)

In [ ]:
data.head()

In [ ]:
#this is gets the sum of the donations, but we also need the number
donsum = data.groupby(['ID', 'Name']).sum()

In [ ]:
donsum

In [ ]:
donsum.reset_index(inplace= True)

In [ ]:
donsum.head()

In [ ]:
donsum = pd.DataFrame(donsum)

In [ ]:
donsum.head()

In [ ]:
doncounts = data.ID.value_counts().reset_index()

In [ ]:
doncounts.rename(columns={'index':'ID','ID':'Donation_Count'}, inplace = True)

In [ ]:
doncounts.head()

In [ ]:
doncounts.shape

In [ ]:
donsum.shape

In [ ]:
count_amount = doncounts.merge(donsum, how = 'inner', on = 'ID')

In [ ]:
count_amount.head()

In [ ]:
data.rename(columns={'Recognition Lookup ID':'ID'},inplace = True)

In [ ]:
data.columns

In [ ]:
count_amount.columns

In [ ]:
count_amount = pd.DataFrame(count_amount)

In [ ]:
count_amount.head()

In [ ]:
nameID = pd.DataFrame(data[["ID","Name"]])

In [ ]:
nameID.head()

In [ ]:
#clusterdata = count_amount.merge(nameID, how = 'inner', on = 'ID')

In [ ]:
type(nameID.ID)

In [ ]:
type(count_amount.ID)

In [ ]:
count_amount.drop(columns = "Recognition Amount", inplace = True)

In [ ]:
count_amount.head()

In [ ]:
newdata = count_amount.merge(nameID, on= 'ID', how= 'left')

In [ ]:
count_amount.head()

In [ ]:
nameID.head()

In [ ]:
nameID.drop_duplicates(subset='ID', inplace = True)

In [ ]:
nameID.head()

In [ ]:
newdata = count_amount.merge(nameID, on= 'ID', how= 'left') #lets try the merge again after dropping duplicates

In [ ]:
newdata.head(10)

In [ ]:
#newdata.set_index(['ID','Name'], inplace = True)

In [ ]:
newdata.head(12)

In [ ]:
count_amount = count_amount.set_index(['ID','Name'])

In [ ]:
#count_amount.astype('int32').dtypes This is precisely what I need: two dimmensions of donor contribution for clustering
count_amount.head()

In [ ]:
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

In [ ]:
datascaled = preprocessing.scale(count_amount)

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,14))

clusdata = visualizer.fit(datascaled) # Fit the data to the visualizer 
clusdata.show()

In [ ]:
cluster_value = clusdata.elbow_value_

In [ ]:
cluster_value

In [ ]:
kmeans = KMeans(n_clusters=cluster_value)

In [ ]:
kmeans.fit(datascaled)

In [ ]:
count_amount['Cluster'] = kmeans.labels_.astype(int)

In [ ]:
count_amount.head()

In [ ]:
round(count_amount.groupby(['Cluster']).mean())

In [ ]:
round(count_amount.groupby(['Cluster']).std()) 

In [ ]:
plot = plt.scatter(count_amount.Amount,count_amount.Donation_Count,c=count_amount.Cluster,cmap='gnuplot')

plt.xlabel('Donation Amount In Dollars')
plt.ylabel('Donation Volume')
plt.title('Donor Clusters')
plt.plot()
plt.show()

In [ ]:
#lets go revisit the progressors data
progressors.head()

In [ ]:
progressors.drop('Recognition Amount', axis = 1, inplace = True)

In [ ]:
#creating a breakdown/groupby of donors based on different aggregations of their donations
groupedprog = progressors.groupby(['Recognition Lookup ID', 'Recognition Name']).agg([np.sum, np.mean, np.std, np.ma.count])

In [ ]:
groupedprog.head(14)

In [ ]:
from pandas import ExcelWriter

In [ ]:
#with ExcelWriter('donor_segments_2020_03_11.xlsx') as writer:
    #groupedprog.to_excel(writer, sheet_name='prime_donors')
    #count_amount.to_excel(writer, sheet_name='donor_clusters')

In [ ]:
# Now as per Brittany's request, looking for donors who have donated in the range of 50 to 1000 dollars exclusively

In [ ]:
data.columns

In [ ]:
#subsetting the data based on a range of donation amount as a good indication of donors who will progress
newnotrange = data[(data['Recognition Amount'] < 50) | (data['Recognition Amount'] > 1000)]

In [ ]:
newnotrange.head()

In [ ]:
outID = newnotrange.ID.unique()

In [ ]:
pd.Series(outID);

In [ ]:
newrange = data[~data.ID.isin(outID)] #what this is doing is excluding IDs who were outside the range from above

In [ ]:
newrange.columns

In [ ]:
newrangegrouped = newrange.groupby(['ID','Name'])

In [ ]:
#A new aggregation on the subsetted data
newrange = pd.DataFrame(newrangegrouped['Amount'].agg([np.sum, np.mean, np.std, np.ma.count]))

In [ ]:
newrange.head(7)

In [ ]:
newrange.to_excel('50to1000donors.xlsx')